# Pyparsing
Když potřebujeme z textového řetězce dostat nějakou informaci, můžeme použít základní metody na textové řetězce napojené - např. *split* či *find*. Tento přístup se ale jen pro lehce komplikovanější úlohy stává neprůchodným. Proto vyvstane potřeba používat regulární výrazy. V Pythonu se obvykle použije implementace z balíčku *re* (zájemci mohou najít v tomto repozitáří povídání o základech jeho používání). Bohužel regulární výrazy mají tendenci stát se velice nepřehlednými.  
Jedním z možných řešení je používat gramatiky, konrétně v případě Pythonu balíček Pyparsing. Ten podporuje rozpad patternů na jednodušší prvky. Navíc defautlně ignoruje bílé znaky, což opět činí patterny přehlednější. Nicméně v případě potřeby se dá toto ignorování bílých znaků modifikovat.  

Pozn.: ujistěte se, že máte nainstalovanou trojkovou verzi pyparsingu. Jinak byste se mohli setkat s chybami typu
```
AttributeError: 'And' object has no attribute 'parse_string'
```

### Jak na parsování

In [2]:
import pyparsing as pp

Začněme s něčím jednoduchým. Mějme sqlkový select, ze kterého chceme dostat jak jméno poptávaného sloupce, tak jméno zdrojové tabulky. Select je opravdu ten nejjednodušší možný, tj. pracujeme zde s konstrukcí
```sql
select sloupec from tabulka;
```
Vidíme, že tu máme dva typy slov (přesněji shluků znaků - na konci dotazu máme středník). První typ slov bude vždy stejný - jedná se o "select", "from" a ";". Při vytváření pyparsingového patternu, tj. šablony, která bude programu říkat, co má vlastně hledat, je napíšeme explicitně. U druhého typu slov se bude obsah měnit - můžeme se totiž ptát na jakýkoli sloupec z jakékoli tabulky. Pro začátek předpokládejme, že názvy budou tvořeny pouze písmeny. Ty bychom sice teoreticky mohli ručně vypisovat, ale praktičtější bude použít již předpřipravenou konstantu *pyparsing.alphas* (resp. *pp.alphas* kvůli aliasu balíčku):

In [3]:
print(f"Data type of pp.alphas: {type(pp.alphas)}")
print(f"Content of pp.alphas: {pp.alphas}")

Data type of pp.alphas: <class 'str'>
Content of pp.alphas: ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


Chybí tu sice například české znaky, ale počítejme s tím, že ty nikdo do názvů sloupců/tabulek nedává.  
Samotná slova či přesněji shluky znaků obklopených znaky bílými vytvoříme provoláním *pyparsing.Word*. Do kulatých závorkách při vytváření tohoto objektu vložíme znaky, ze kterých se ono "slovo" může skládat. V našem případě tam patří právě textový řetězec *pp.alphas*. Pokud bychom z nějakého důvodu potřebovali pouze velká anebo naopak pouze malá písmena, použijeme obyčejné metody vlastní všem stringům - *upper()* a *lower()*.   
Nyní máme vytvořený pattern. Na tomto objektu provoláme metodu *parse_string*, které jako argument předáme string, který chceme prozkoumat. 

In [3]:
query = """
select someCol from someTable;
"""
query_pattern = "select" + pp.Word(pp.alphas) + "from" + pp.Word(pp.alphas) + ";"
result = query_pattern.parse_string(query)
result

ParseResults(['select', 'someCol', 'from', 'someTable', ';'], {})

Všimněme si, že výsledek je stejný, i když jsou ve zkoumaném textu i jiné bílé znaky než mezerník:

In [4]:
query = """
select\tsomeCol



from someTable;
"""
query_pattern = "select" + pp.Word(pp.alphas) + "from" + pp.Word(pp.alphas) + ";"
result = query_pattern.parse_string(query)
result

ParseResults(['select', 'someCol', 'from', 'someTable', ';'], {})

Může se stát, že některá část patternu pro nás bude nezajímavá. V parsování musí být uplatněná, ale ve výpisu výsledků by nám jenom překážela. V takovém případě můžeme použít *pp.Suppress*. Vidíme, že po jeho aplikaci na středník se tento znak už ve výsledku neobjeví.

In [4]:
query = """
select someCol from someTable;
"""
query_pattern = "select" + pp.Word(pp.alphas) + "from" + pp.Word(pp.alphas) + pp.Suppress(";")
result = query_pattern.parse_string(query)
result

ParseResults(['select', 'someCol', 'from', 'someTable'], {})

No jo, co když ale spíše než vyhození nedůležitých prvků z výsledku chceme důležité prvky vybrat? Ano, mohli bychom suppress dát na téměř všechno v patternu, takové řešení by ale moc elegantní nebylo. Naštěstí existuje vhodnější postup. První věc, kterou uděláme, je vložení *pp.Word* do samostatných proměnných. Následně na ně napojíme jména s pomocí metody *set_result_name*.

In [6]:
query = """
select someCol from someTable;
"""

column_name = pp.Word(pp.alphas).set_results_name("columns")
table_name = pp.Word(pp.alphas).set_results_name("tables")
query_pattern = "select" +  column_name + "from" + table_name + ";"
result = query_pattern.parse_string(query)
result

ParseResults(['select', 'someCol', 'from', 'someTable', ';'], {'columns': 'someCol', 'tables': 'someTable'})

Vidíme, že ve výsledném objektu se krom listu plní i druhá část - slovník. Do něj se nemusíme dostávat indexací ParseResult objektu, přístup je možný přímo:

In [7]:
result["columns"]

'someCol'

In [8]:
result["tables"]

'someTable'

#### Možnosti Word objektů
Podívejme se na pár možností, jak *Word* objekty obohatit. Dejme tomu, že by se v názvech sloupců a tabulek objevila podtržítka. V takovém případě parsování selže. Naštěstí chybová hláška napoví, kde vznikl problém:

In [9]:
query = """
select some_col from some_table;
"""

column_name = pp.Word(pp.alphas).set_results_name("columns")
table_name = pp.Word(pp.alphas).set_results_name("tables")
query_pattern = "select" +  column_name + "from" + table_name + ";"
result = query_pattern.parse_string(query)
result

ParseException: Expected 'from', found '_'  (at char 12), (line:2, col:12)

Podívejme se o pár řádků výše - uvidíme, že *pp.alphas* je fakticky obyčejný textový řetězec. Tudíž nic nebrání k němu operátorem plus přidat další znak:

In [10]:
query = """
select some_col from some_table;
"""

column_name = pp.Word(pp.alphas + "_").set_results_name("columns")
table_name = pp.Word(pp.alphas + "_").set_results_name("tables")
query_pattern = "select" +  column_name + "from" + table_name + ";"
result = query_pattern.parse_string(query)
print(f"Tabulky: {result['tables']}, sloupce: {result['columns']}")

Tabulky: some_table, sloupce: some_col


Podobná situace nastane, pokud názvy sloupců a tabulek budou obsahovat číslovky. Mohli bychom sice explicitně psát 
```
pp.alphas + "_" + "0123456789"
```
ale pohodlnější bude použít *pp.nums*, anebo rovnou *pp.alphanums*, které v sobě už *pp.alphas* a *pp.nums* spojuje.

In [11]:
query = """
select some_col_1 from some_table_1;
"""

column_name = pp.Word(pp.alphas + "_" + pp.nums).set_results_name("columns")
table_name = pp.Word(pp.alphanums + "_").set_results_name("tables")
query_pattern = "select" +  column_name + "from" + table_name + ";"
result = query_pattern.parse_string(query)
print(f"Tabulky: {result['tables']}, sloupce: {result['columns']}")

Tabulky: some_table_1, sloupce: some_col_1


Mohlo by se stát, že v názvech sice číslovky povolené jsou, nesmějí se ale objevit na začátku. Takovéto pravidlo vynucuje například python u jmen proměnných. Pokud něco takového chceme implementovat, musíme *pp.Words* provolat s dvěma parametry. První bude obsahovat znaky povolené pro první znak hledaných slov, druhý parametr pak znaky pro zbylou část slov. Pakliže druhý parametr uveden není, aplikuje se parametr první na slova celá. Tak to koneckonců bylo u předchozích kusů kódu. Nyní si ukažme dvouparametrové provolání *pp.Word* na příkladech. První bude obsahovat validní query, druhý pak nevhodnou query s číslovkama na začátku názvu sloupce.

In [12]:
query = """
select some_col_1 from some_table_1;
"""

column_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("columns")
table_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("tables")
query_pattern = "select" +  column_name + "from" + table_name + ";"
result = query_pattern.parse_string(query)
print(f"Tabulky: {result['tables']}, sloupce: {result['columns']}")

Tabulky: some_table_1, sloupce: some_col_1


In [13]:
query = """
select 1_some_col from 1_some_table;
"""

column_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("columns")
table_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("tables")
query_pattern = "select" +  column_name + "from" + table_name + ";"
result = query_pattern.parse_string(query)
print(f"Tabulky: {result['tables']}, sloupce: {result['columns']}")

ParseException: Expected W:(A-Z_a-z, 0-9A-Z_a-z), found '1'  (at char 8), (line:2, col:8)

Moh by se vyskytnout případ, kdy bychom chtěli namatchovat slovo o předem stanovené velikosti. Představme si například, že u tabulky použijeme třípísmenný alias. Tehdy se použije další nepovinný parametr konstruktoru *Word* a to konkrétně *exact*. Ten se položí roven chtěnému počtu znaků. Pokud bychom chtěli mít *interval* počtu znaků, využijeme namísto toho parametry *min* a *max*.

In [14]:
query = """
select sta.some_col_1 from some_table_1 sta;
"""

column_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("columns")
table_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("tables")
table_alias = pp.Word(pp.alphanums, exact=3).set_results_name("alias")

query_pattern = "select" +  table_alias + "." + column_name + "from" + table_name + table_alias + ";"
result = query_pattern.parse_string(query)
print(f"Tabulky: {result['tables']} (alias {result['alias']}), sloupce: {result['columns']}")

Tabulky: some_table_1 (alias sta), sloupce: some_col_1


#### Combine

No jo, nyní by ale prošla i nevalidní query, která má mezi aliasem, tečkou a jménem sloupce mezery.

In [15]:
query = """
select sta  .  some_col_1 from some_table_1 sta;
"""

column_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("columns")
table_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("tables")
table_alias = pp.Word(pp.alphanums, exact=3).set_results_name("alias")

query_pattern = "select" +  table_alias + "." + column_name + "from" + table_name + table_alias + ";"
result = query_pattern.parse_string(query)
print(f"Tabulky: {result['tables']} (alias {result['alias']}), sloupce: {result['columns']}")

Tabulky: some_table_1 (alias sta), sloupce: some_col_1


V takovýchto případech je nutné slepit jednotlivé části dohromady pomocí *Combine*. Zdůrazněme, že v pozadí se všechny zkombinované části patternu slepí do jednoho textového řetězce. Tj. nejde tu pouze o shluknutí částí, jak se dá dosáhnout v o něco níže ukázaní *pp.Group*.  
Nejprve příklad, jak Combine zabrání naparsování špatné query:

In [20]:
query = """
select sta  .  some_col_1 from some_table_1 sta;
"""

column_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("columns")
table_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("tables")
table_alias = pp.Word(pp.alphanums, exact=3).set_results_name("alias")
alias_with_column = pp.Combine(table_alias + "." + column_name)

query_pattern = "select" +  alias_with_column + "from" + table_name + table_alias + ";"
result = query_pattern.parse_string(query)
print(f"Tabulky: {result['tables']} (alias {result['alias']}), sloupce: {result['columns']}")

ParseException: Expected '.', found ' '  (at char 11), (line:2, col:11)

A nyní si ukážeme, jak ten samý kód funguje u validní query:

In [24]:
query = """
select sta.some_col_1 from some_table_1 sta;
"""

column_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("columns")
table_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("tables")
table_alias = pp.Word(pp.alphanums, exact=3).set_results_name("alias")
alias_with_column = pp.Combine(table_alias + "." + column_name)

query_pattern = "select" +  alias_with_column + "from" + table_name + table_alias + ";"
result = query_pattern.parse_string(query)
print(f"Tabulky: {result['tables']} (alias {result['alias']}), sloupce: {result['columns']}")

Tabulky: some_table_1 (alias sta), sloupce: some_col_1


#### Nepovinné a vícenásobné části patternu

Když bychom chtěli nyní kódem naparsovat query bez aliasů, tak parsování selže. Nicméně query správně napsaná je. Musíme tudíž parsovadlu nějak říci, že aliasy jsou nepovinné. Toho dosáhneme jejich vložením do *pp.Optional*.

In [25]:
query = """
select some_col_1 from some_table_1;
"""

column_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("columns")
table_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("tables")
table_alias = pp.Word(pp.alphanums, exact=3).set_results_name("alias")
alias_with_column = pp.Combine(pp.Optional(table_alias + ".") + column_name)

query_pattern = "select" +  alias_with_column + "from" + table_name + pp.Optional(table_alias) + ";"
result = query_pattern.parse_string(query)

if "alias" in result.keys():
    print(f"Tabulky: {result['tables']} (alias {result['alias']}), sloupce: {result['columns']}")
else:
    print(f"Tabulky: {result['tables']} (no alias), sloupce: {result['columns']}")

Tabulky: some_table_1 (no alias), sloupce: some_col_1


Ptát se prostřednictvím selectu jen na jeden sloupec je docela nuda. Nicméně aby pattern rozšířenou query zchroustal, musíme do něj přidat *pp.ZeroOrMore*. Do tohoto nového objektu přidáme čárku a sloupec.V případě, kdy bychom z nějakého důvodu potřebovali aspoň jeden výskyt (zde to nedává smysl kvůli čárce), existuje i *pp.OneOrMore*.  
Když se ale podíváme na výsledný výpis, vidíme v něm jen poslední sloupec.

In [26]:
query = """
select some_col_1, some_col_2, some_col_3 from some_table_1;
"""

column_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("columns")
table_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("tables")
table_alias = pp.Word(pp.alphanums, exact=3).set_results_name("alias")
alias_with_column = pp.Combine(pp.Optional(table_alias + ".") + column_name)

query_pattern = (
    "select" +  alias_with_column 
    + pp.ZeroOrMore(pp.Suppress(",") + alias_with_column) 
    + "from" + table_name + pp.Optional(table_alias) + ";"
)
result = query_pattern.parse_string(query)

if "alias" in result.keys():
    print(f"Tabulky: {result['tables']} (alias {result['alias']}), sloupce: {result['columns']}")
else:
    print(f"Tabulky: {result['tables']} (no alias), sloupce: {result['columns']}")

Tabulky: some_table_1 (no alias), sloupce: some_col_3


Když si necháme vypsat obsah result objektu, uvidíme v něm všechny tři sloupce jen v první "listové" části. Co teď s tím? Přeci jen použití slovníku podobné syntaxe vypadalo na začátku jako docela velké ulehčení práce.

In [27]:
result

ParseResults(['select', 'some_col_1', 'some_col_2', 'some_col_3', 'from', 'some_table_1', ';'], {'columns': 'some_col_3', 'tables': 'some_table_1'})

Řešení je vcelku jednoduché - stačí přidat do metody *set_results_name* parametr *list_all_matches* s hodnotou True.

In [28]:
query = """
select some_col_1, some_col_2, some_col_3 from some_table_1;
"""

column_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("columns", list_all_matches=True)
table_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("tables", list_all_matches=True)
table_alias = pp.Word(pp.alphanums, exact=3).set_results_name("alias")
alias_with_column = pp.Combine(pp.Optional(table_alias + ".") + column_name)

query_pattern = (
    "select" +  alias_with_column 
    + pp.ZeroOrMore("," + alias_with_column) 
    + "from" + table_name + pp.Optional(table_alias) + ";"
)
result = query_pattern.parse_string(query)

if "alias" in result.keys():
    print(f"Tabulky: {result['tables']} (alias {result['alias']}), sloupce: {result['columns']}")
else:
    print(f"Tabulky: {result['tables']} (no alias), sloupce: {result['columns']}")

Tabulky: ['some_table_1'] (no alias), sloupce: ['some_col_1', 'some_col_2', 'some_col_3']


#### OR logika

Mějme situaci, kdy bychom chtěli současně parsovat selecty a současně delety. Fakticky si tak musíme vytvořit jednak pattern pro select, jednak pattern pro delete. Mezi nimi posléze nastavíme OR pomocí znaku pipy (|, tj. na české klávesnici pravý alt + W).

In [29]:
query_select = """
select some_col_1, some_col_2, some_col_3 from some_table_1;
"""

query_delete = """
delete from some_table_1;
"""

column_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("columns", list_all_matches=True)
table_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("tables", list_all_matches=True)
table_alias = pp.Word(pp.alphanums, exact=3).set_results_name("alias")
alias_with_column = pp.Combine(pp.Optional(table_alias + ".") + column_name)

select_query_pattern = (
    "select" +  alias_with_column 
    + pp.ZeroOrMore("," + alias_with_column) 
    + "from" + table_name + pp.Optional(table_alias) + ";"
)

delete_query_pattern = (
    "delete from" + table_name + ";"
)

query_pattern = (select_query_pattern | delete_query_pattern)

result = query_pattern.parse_string(query_select)

if "alias" in result.keys():
    print(f"Tabulky: {result['tables']} (alias {result['alias']}), sloupce: {result['columns']}")
else:
    print(f"Tabulky: {result['tables']} (no alias), sloupce: {result['columns']}")
    
result = query_pattern.parse_string(query_delete)

if "alias" in result.keys():
    print(f"Tabulky: {result['tables']} (alias {result['alias']})")
else:
    print(f"Tabulky: {result['tables']} (no alias)")

Tabulky: ['some_table_1'] (no alias), sloupce: ['some_col_1', 'some_col_2', 'some_col_3']
Tabulky: ['some_table_1'] (no alias)


Nyní se pokusíme vytvořit jednoduchou where podmínku. Klíčová slova se nám nyní začínají hromadit a tak bude lepší je vypisovat nikoli jako textové řetězce, ale jako pyparsingové objekty. konkrétně použijeme tzv. *CaselessKeyword*. Caseless znamená, že se nám provede match nezávisle na velikosti písmen. Keyword značí, že nalezené klíčové slovo musí být následované neklíčovým slovem. Tím se liší od podobného objektu *Literal*, který by třeba pro slovo "if" namatchoval začátek "ifAndOnlyIf".  
Ve wheru budeme porovnávat hodnotu sloupce s nějakou číselnou hodnotou. Nicméně porovnávacích operátorů je několik. Mohli bychom sice použít OR konstrukci, ale mnohem pohodlnější bude využití *one_of* funkce. Ta přebere textový řetězec a roztrhá ho podle bílých znaků. Výsledek se pak bude matchovat na libovolný z takto vytvořených tokenů.

In [31]:
query = """
select sta.some_col_1 
from some_table_1 sta 
where sta.some_col_2 >= 5
and sta.some_col_3 = 50
;
"""
select_keyword = pp.CaselessKeyword("select")
from_keyword = pp.CaselessKeyword("from")
where_keyword = pp.CaselessKeyword("where")
and_keyword = pp.CaselessKeyword("and")
or_keyword = pp.CaselessKeyword("or")

column_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("columns", list_all_matches=True)
table_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("tables", list_all_matches=True)
table_alias = pp.Word(pp.alphanums, exact=3).set_results_name("alias")
alias_with_column = pp.Combine(pp.Optional(table_alias + ".") + column_name)

comparison = pp.one_of("= < > >= <= <>")
one_cond = alias_with_column + comparison + (pp.Word(pp.alphas) | pp.Word(pp.nums + ",."))
where_condition = where_keyword + one_cond + pp.ZeroOrMore((and_keyword|or_keyword) + one_cond )

select_query_pattern = (
    select_keyword +  alias_with_column 
    + pp.ZeroOrMore("," + alias_with_column) 
    + from_keyword + table_name + pp.Optional(table_alias) 
    + pp.Optional(where_condition)
    + ";"
)

result = select_query_pattern.parse_string(query)
if "alias" in result.keys():
    print(f"Tabulky: {result['tables']} (alias {result['alias']}), sloupce: {result['columns']}")
else:
    print(f"Tabulky: {result['tables']} (no alias), sloupce: {result['columns']}")

Tabulky: ['some_table_1'] (alias sta), sloupce: ['some_col_1', 'some_col_2', 'some_col_3']


#### Rekurze (select v selectu)

Představme si ale jinou situaci - co kdyby byl ve where podmínce subselect? Tak by se nám vytvořila rekurze, kdy by předpis patternu pro select odkazoval sám na sebe. To v pyparsingu vytvořit lze, ale musí se na to použít speciální konstrukce.  
Nejprve se element s rekurzí položí rovný **pp.Forward()**. Posléze se do něj vloží celá definice pomocí operátoru "<<".  
Všimněte si, že jsme celý select (až na středník) obalili do *pp.Group*. Díky tomu select drží pohromadě a nedojde k pomíchání jednotlivých úrovní selectu a subselectu. V result objektu nejsou tabulky a sloupce subselectu - k nim se musíme dostat tak, že nejprve skočíme do výsledku zodpovědného právě za onen subslelect.

In [32]:
query = """
select sta.some_col_1 
from some_table_1 sta 
where sta.some_col_2 in (select ant.some_another_col from another_table ant)
;
"""

select_keyword = pp.CaselessKeyword("select")
from_keyword = pp.CaselessKeyword("from")
where_keyword = pp.CaselessKeyword("where")
and_keyword = pp.CaselessKeyword("and")
or_keyword = pp.CaselessKeyword("or")
in_keyword = pp.CaselessKeyword("in")

column_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("columns", list_all_matches=True)
table_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("tables", list_all_matches=True)
table_alias = pp.Word(pp.alphanums, exact=3).set_results_name("alias")
alias_with_column = pp.Combine(pp.Optional(table_alias + ".") + column_name)

select_fragment = pp.Forward()

select_fragment << pp.Group(
    select_keyword +  alias_with_column 
    + pp.ZeroOrMore("," + alias_with_column) 
    + from_keyword + table_name + pp.Optional(table_alias) 
    + pp.Optional(
        where_keyword + alias_with_column + in_keyword + "(" 
        +  select_fragment.set_results_name("inner_select") 
        + ")"
    )
)

query_pattern = select_fragment + pp.Suppress(";")

result = query_pattern.parse_string(query)[0]

print("For main query:")
if "alias" in result.keys():
    print(f"Tabulky: {result['tables']} (alias {result['alias']}), sloupce: {result['columns']}")
else:
    print(f"Tabulky: {result['tables']} (no alias), sloupce: {result['columns']}")
    
print("For subquery:")
subquery_result = result["inner_select"]
if "alias" in subquery_result.keys():
    print(f"Tabulky: {subquery_result['tables']} (alias {subquery_result['alias']}), sloupce: {subquery_result['columns']}")
else:
    print(f"Tabulky: {subquery_result['tables']} (no alias), sloupce: {subquery_result['columns']}")

For main query:
Tabulky: ['some_table_1'] (alias sta), sloupce: ['some_col_1', 'some_col_2']
For subquery:
Tabulky: ['another_table'] (alias ant), sloupce: ['some_another_col']


TODO: 
-  přidat ignore
-  přidat negující vlnku

#### Užitečné metody patternu
Doposud jsme k parsování textu používali metodu patternu *parse_string*. Ta se defautlně snaží na text namatchovat pattern. Pokud text pokračuje dále, to už *parse_string* nijak netrápí.

In [33]:
query = """
select some_col_1 from some_table_1;
blabla
"""

column_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("columns")
table_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("tables")
query_pattern = "select" +  column_name + "from" + table_name + ";"
result = query_pattern.parse_string(query)
print(f"Tabulky: {result['tables']}, sloupce: {result['columns']}")

Tabulky: some_table_1, sloupce: some_col_1


Pokud ale takové chování není žádoucí, lze do *parse_string* přidat parametr *parse_all* s hodnotou "all". Po takové změně výše uvedený příkaz skončí chybou:

In [34]:
query = """
select some_col_1 from some_table_1;
blabla
"""

column_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("columns")
table_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("tables")
query_pattern = "select" +  column_name + "from" + table_name + ";"
result = query_pattern.parse_string(query, parse_all=True)
print(f"Tabulky: {result['tables']}, sloupce: {result['columns']}")

ParseException: Expected end of text, found 'blabla'  (at char 38), (line:3, col:1)

Často budeme parsovat soubory. Ty bychom sice mohli otevřít a poté prohnat skrze *parse_string*, ale metoda *parse_file* nám situaci zjednodušuje - obě operace v sobě totiž spojuje.

In [35]:
column_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("columns")
table_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("tables")
query_pattern = "select" +  column_name + "from" + table_name + ";"
result = query_pattern.parse_file("file_for_pyparsing.txt")
print(f"Tabulky: {result['tables']}, sloupce: {result['columns']}")

Tabulky: some_table_1, sloupce: some_col_1


Někdy se pro nás zajímavé části textu nalézají zahrabané v hromadě hlušiny, pro kterou gramatiku vymýšlet nechceme. Tehdy nám poslouží metoda *scan_string*. U ní sbudeme muset upravit práci v výsledným objektem. Jím je totiž generátor (proto procházení forem), který poskytuje jednočlenné tuply s výsledky (proto [0]). Všiměte si, že *scan_string* nalezne všechny (nepřekrývající se) výskyty patternu v textu.

In [36]:
query = """
blabla
select some_col_1 from some_table_1;
blabla
select some_col_2 from some_table_2;
tralalala
"""

column_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("columns")
table_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("tables")
query_pattern = "select" +  column_name + "from" + table_name + ";"
results = query_pattern.scan_string(query)
for one_result in results:
    print(f"Tabulky: {one_result[0]['tables']}, sloupce: {one_result[0]['columns']}")

Tabulky: some_table_1, sloupce: some_col_1
Tabulky: some_table_2, sloupce: some_col_2


Práce s výstupem *scan_string* je poněkud nešikovná. Proto byl vytvořena metoda *search_string*, která funguje velice podobně, akorát namísto generátoru vrací listu podobný objekt, jehož prvky nejsou tuply, ale samotné ParseResulty (tj. věci, se kterou jsme pracovali u výstupu *parse_string*). Fakticky to znamená, že ve zpracování výsledku už můžeme indexaci pominout.

In [37]:
query = """
blabla
select some_col_1 from some_table_1;
blabla
select some_col_2 from some_table_2;
tralalala
"""

column_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("columns")
table_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("tables")
query_pattern = "select" +  column_name + "from" + table_name + ";"
results = query_pattern.search_string(query)
for one_result in results:
    print(f"Tabulky: {one_result['tables']}, sloupce: {one_result['columns']}")

Tabulky: some_table_1, sloupce: some_col_1
Tabulky: some_table_2, sloupce: some_col_2


Když člověk rozšiřuje pattern a zkouší ho na nových příkladech, může se stát, že díky takto vzniklým změnám už pattern nebude sedět na příklady staré. To je dobré podchytit v nějaké testovací funkci či metodě, která spustí naráz hromadu příkladů. Tato metoda se v pyparsingu nazývá *run_tests*. Jako parametr přebírá list se seznamem příkladů. Návratovou hodnotou je tuple o dvou prvcích. V prvním je boolean indukující, zda všechny testy dopadly úspěšně, v druhém pak ParseResulty pro každý z testů.

In [38]:
test_queries = [
    "select some_col_1 from some_table_1;",
    "select sta.some_col_1 from some_table_1 sta;",
    "select tra.sta.some_col_1 from some_table_1 sta tra;",
]

column_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("columns", list_all_matches=True)
table_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("tables", list_all_matches=True)
table_alias = pp.Word(pp.alphanums, exact=3).set_results_name("alias")
alias_with_column = pp.Combine(pp.Optional(table_alias + ".") + column_name)

query_pattern = (
    "select" +  alias_with_column 
    + pp.ZeroOrMore("," + alias_with_column) 
    + "from" + table_name + pp.Optional(table_alias) + ";"
)

query_pattern.run_tests(test_queries)


select some_col_1 from some_table_1;
['select', 'some_col_1', 'from', 'some_table_1', ';']
- columns: ['some_col_1']
- tables: ['some_table_1']

select sta.some_col_1 from some_table_1 sta;
['select', 'sta.some_col_1', 'from', 'some_table_1', 'sta', ';']
- alias: 'sta'
- columns: ['some_col_1']
- tables: ['some_table_1']

select tra.sta.some_col_1 from some_table_1 sta tra;
select tra.sta.some_col_1 from some_table_1 sta tra;
              ^
ParseException: Expected 'from', found '.'  (at char 14), (line:1, col:15)
FAIL: Expected 'from', found '.'  (at char 14), (line:1, col:15)


(False,
 [('select some_col_1 from some_table_1;',
   ParseResults(['select', 'some_col_1', 'from', 'some_table_1', ';'], {'columns': ['some_col_1'], 'tables': ['some_table_1']})),
  ('select sta.some_col_1 from some_table_1 sta;',
   ParseResults(['select', 'sta.some_col_1', 'from', 'some_table_1', 'sta', ';'], {'alias': 'sta', 'columns': ['some_col_1'], 'tables': ['some_table_1']})),
  ('select tra.sta.some_col_1 from some_table_1 sta tra;',
   Expected 'from', found '.'  (at char 14), (line:1, col:15))])

Alternativně může být vstupem *run_tests* i víceřádkový textový řetězec, kde jsou jednotlivé testy odděleny hashtagem a komentářem.

In [39]:
test_queries_string = """
    # komenter k prvni query
    select some_col_1 from some_table_1;
    # komenter k druhe query
    select sta.some_col_1 from some_table_1 sta;
    # komenter k treti query
    select tra.sta.some_col_1 from some_table_1 sta tra;
"""

column_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("columns", list_all_matches=True)
table_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("tables", list_all_matches=True)
table_alias = pp.Word(pp.alphanums, exact=3).set_results_name("alias")
alias_with_column = pp.Combine(pp.Optional(table_alias + ".") + column_name)

query_pattern = (
    "select" +  alias_with_column 
    + pp.ZeroOrMore("," + alias_with_column) 
    + "from" + table_name + pp.Optional(table_alias) + ";"
)

query_pattern.run_tests(test_queries_string)


# komenter k prvni query
select some_col_1 from some_table_1;
['select', 'some_col_1', 'from', 'some_table_1', ';']
- columns: ['some_col_1']
- tables: ['some_table_1']

# komenter k druhe query
select sta.some_col_1 from some_table_1 sta;
['select', 'sta.some_col_1', 'from', 'some_table_1', 'sta', ';']
- alias: 'sta'
- columns: ['some_col_1']
- tables: ['some_table_1']

# komenter k treti query
select tra.sta.some_col_1 from some_table_1 sta tra;
select tra.sta.some_col_1 from some_table_1 sta tra;
              ^
ParseException: Expected 'from', found '.'  (at char 14), (line:1, col:15)
FAIL: Expected 'from', found '.'  (at char 14), (line:1, col:15)


(False,
 [('select some_col_1 from some_table_1;',
   ParseResults(['select', 'some_col_1', 'from', 'some_table_1', ';'], {'columns': ['some_col_1'], 'tables': ['some_table_1']})),
  ('select sta.some_col_1 from some_table_1 sta;',
   ParseResults(['select', 'sta.some_col_1', 'from', 'some_table_1', 'sta', ';'], {'alias': 'sta', 'columns': ['some_col_1'], 'tables': ['some_table_1']})),
  ('select tra.sta.some_col_1 from some_table_1 sta tra;',
   Expected 'from', found '.'  (at char 14), (line:1, col:15))])

Nakonec i ukažme metodu, která vytváří graf patternu. Pro její použití bude potřeba napřed nainstalovat dodatečný balíček a to pomocí příkazu
```
pip install pyparsing[diagrams]
```
Nutno poznamenat, že vytvářeč grafu občas selže (například u patternu, který jsme použil pro demonstaci ORu)

In [40]:
query = """
select some_col_1 from some_table_1;
"""

column_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("columns")
table_name = pp.Word(pp.alphas + "_", pp.alphanums + "_").set_results_name("tables")
query_pattern = "select" +  column_name + "from" + table_name + ";"

query_pattern.create_diagram("soubor.html", show_results_names=True, show_groups=True)